In [ ]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

from pipeline.metrics import accuracy_b
from pipeline.model_methods import validate, train
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset

from efficientnet_pytorch import EfficientNet

import timm


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
temp_y_train = pd.read_json("../../dataset/FAKE/metadata.json")

y_train_label = list()
y_train_original = list()
for key in temp_y_train.keys():
    temp_y_train[key]["filename"] = key

    
#len(y_train[y_train.label == "FAKE"]), len(y_train[y_train.label == "REAL"])
# len(y_train[test])
#y_train = pd.read_csv(r'data\metadata.csv')
# len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

In [ ]:
y_train

In [ ]:
key=list(y_train.keys())[0]
y_train[key]["file"] = key

In [ ]:
for key in y_train.keys():
    y_train[key]["file"] = key

In [ ]:
batch_size = 32

x_train = load_img_dataset(r'data\train', batch_size, resize=None, crop=224, num_samples=100000)
x_val = load_img_val_dataset(r'data\img_val', batch_size)

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)

#model = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
#model.classifier = nn.Linear(model.classifier.in_features, 1)

model = model.to(device)

In [ ]:
epochs = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [ ]:
train(model, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device,
             epochs=epochs,
             batch_size=batch_size, 
             useScheduler=True,
             checkpoint=0.35
            )